In [34]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.constants import calorie, R

In [84]:
def nrtl(alpha, tau, x):
    '''
    NRTLによる活量係数の計算

    Parameters
    ----------
    alpha : ndarray(n,n)
        Array of NRTL nonrandomness parameters. n = the number of 
        components in the system.
    tau : ndarray(n,n)
        Array of NRTL tau parameters. tau[i,i] should be set to 0.
    t : float
        Temperature (K)
    x : ndarray(n,)
        Mole fraction of each component

    Returns
    -------
    gamma : ndarray(n,)
        Activity coefficient of each component    
    '''
    
    G = np.exp(-alpha*tau)
    ncomp = x.shape[0]
    gamma = np.zeros_like(x)
    summ = 0
    
    for i in range(ncomp):
        summ = 0
        for j in range(ncomp):
            summ += x[j]*G[i,j]/np.sum(G[:,j]*x)*(tau[i,j] - (np.sum(x*tau[:,j]*G[:,j])/np.sum(G[:,j]*x)))
        gamma[i] = np.sum(tau[:,i]*G[:,i]*x)/np.sum(G[:,i]*x) + summ
        
        return np.exp(gamma)
    
def opt_test():
    T = 138 + 273.15
    g_ini = np.asarray([[0, 1.0],
                    [1.0, 0]])
    tau_ini = g_ini / (R*T)
    alpha_ini = 0.2
    x_ini = np.asarray([0.9898662364, 0.0101337636])
    gamma_ini = nrtl(alpha_ini, tau_ini, x_ini)
    print(gamma_ini)
    
def equilibrium(pi, x, y, p):
    '''
    低圧の気液平衡関係式から活量係数を求める
    '''
    gamma_lab = (pi * y) / (p * x)
    return gamma_lab

def lab_read(pi):
    '''
    実験データの読み取り
    3atm, 5atm, 8atm, 11.2atmそれぞれの圧力条件でのデータ
    '''
    df_lab = pd.read_csv("/home/studio-lab-user/Molecular_distillation_simulator/table.csv").drop(index=[0, 1]).reset_index().drop(columns="index")
    
    df_atm3 = pd.DataFrame(df_lab.iloc[0:10, 0:4]).set_axis(['No.', 'Liquid phase methanol mole fraction', 
                                                         'Vapor phase methanol mole fraction', 'Temperature [℃]'], axis=1)
    # print(df_atm3)
    df_atm5 = pd.DataFrame(df_lab.iloc[0:11, 4:8]).set_axis(['No.', 'Liquid phase methanol mole fraction', 
                                                         'Vapor phase methanol mole fraction', 'Temperature [℃]'], axis=1)
    # print(df_atm5)
    df_atm8 = pd.DataFrame(df_lab.iloc[12:24, 0:4]).set_axis(['No.', 'Liquid phase methanol mole fraction', 
                                                          'Vapor phase methanol mole fraction', 'Temperature [℃]'], axis=1).reset_index().drop(columns="index")
    # print(df_atm8)
    df_atm11 = pd.DataFrame(df_lab.iloc[12:23, 4:8]).set_axis(['No.', 'Liquid phase methanol mole fraction', 
                                                          'Vapor phase methanol mole fraction', 'Temperature [℃]'], axis=1).reset_index().drop(columns="index")
    # print(df_atm11)
    if pi == 3:
        return df_atm3
    elif pi == 5:
        return df_atm5
    elif pi == 8:
        return df_atm8
    else:
        return df_atm11

In [87]:
df_lab = lab_read(11)
equilibrium()

,No.,Liquid phase methanol mole fraction,Vapor phase methanol mole fraction,Temperature [℃]
0,1,0.038,0.164,177.9
1,2,0.067,0.241,174.7
2,3,0.104,0.338,170.1
3,4,0.171,0.434,165.6
4,5,0.24,0.512,161.7
5,6,0.329,0.583,158.2
6,7,0.443,0.664,154.3
7,8,0.511,0.71,152.1
8,9,0.67,0.804,148
9,10,0.717,0.825,146.9
